# **Graded Challenge 3**

Karel Gideon Anugrah Hutajulu | FTDS HCK-BATCH-004

## I. INTRODUCTION

### Objective

Data Wrangling. As a datascientist my job is to convert raw_data into data that can show useful insight and make the data easier to analyze. Use the `SMART` framework to create realistic and targeted goal for the ecommerce business.

---

### Problem Statement

The Look ecommerce team is trying to figure out their marketing staretgy for Q1-Q3 of 2023. They want to decide if they want to allocate their budget in marketing for the New year Holiday or the Summer Holiday. They also want to know what product category should they market.

---

### Goal Statement


**Specific:** Develop a marketing strategy for Q1-Q3 2023 to increase sales for Look ecommerce by determining the best allocation of the marketing budget between the New Year Holiday and the Summer Holiday, and identifying the most promising product category for marketing.

**Measurable:** The effectiveness of the marketing strategy will be measured by comparing the sales data before and after the marketing campaigns, as well as tracking the performance of the selected product category. We hope to see 5% increase in customer spending/transaction.

**Achievable:** The marketing strategy can be developed and implemented within the given timeframe of Q1-Q3 2023. Past sales and marketing data (Q1-Q3 2022)can be used to inform the decision-making process for the allocation of the marketing budget and the identification of the most promising product category for marketing.

**Relevant:** Developing an effective marketing strategy is relevant to Look ecommerce as it can drive sales and increase brand awareness, which can contribute to the company's growth and profitability.

**Time-bound:** Q1 - Q3 2023



## II. Connect Big Query

In [5]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')


Authenticated


In [6]:
from google.cloud import bigquery

#PANDA LIBRARIES USED FOR DESCRIPTION QUESTION NO.6 AND THE POINT QUESTIONS
import pandas as pd

In [7]:
project_id = 'hacktiv8-karel'
client = bigquery.Client(project=project_id)

## III. DESCRIPTION QUESTION / POIN PENJABARAN

Before answering the questions, we will first use the `Distinct` method to find all the different status and product category. This will help us understand the data more and provide better insights

In [8]:
complete_orders = client.query(
'''
SELECT DISTINCT Status

FROM `bigquery-public-data.thelook_ecommerce.orders`

'''
).to_dataframe()

complete_orders

,Status
0,Shipped
1,Complete
2,Returned
3,Cancelled
4,Processing


In [9]:
complete_orders = client.query(
'''
SELECT DISTINCT product_category

FROM `bigquery-public-data.thelook_ecommerce.inventory_items`

'''
).to_dataframe()

complete_orders

,product_category
0,Tops & Tees
1,Socks & Hosiery
2,Accessories
3,Socks
4,Intimates
5,Fashion Hoodies & Sweatshirts
6,Plus
7,Swim
8,Outerwear & Coats
9,Sleep & Lounge


### 1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?


In [10]:
complete_orders = client.query(
'''
#Extract Year and month from column 'created_at' that has DATETIME type
SELECT extract(year from created_at) as year,
       extract(month from created_at) as month, 
#Count the amount of orders
       count(order_id) as order_amount

FROM `bigquery-public-data.thelook_ecommerce.orders`
#add parameter where status is complete and the dates are in Q1-Q3
WHERE status='Complete' and created_at BETWEEN '2022-01-01' AND '2022-10-01'
GROUP BY year,month
ORDER BY year,month ASC
'''
).to_dataframe()

#Display dataframe
complete_orders

,year,month,order_amount
0,2022,1,820
1,2022,2,800
2,2022,3,946
3,2022,4,927
4,2022,5,1022
5,2022,6,1086
6,2022,7,1308
7,2022,8,1317
8,2022,9,1466


My Analysis: 



---

We can see a steady upwards increase in total orders from Q1 to Q2 of 2023. This can be caused by increase in customer base of the ecommerce website or the frequency of orders made by existing customer.

### 2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?

In [11]:
orders_value = client.query(
'''
#Extract Year and month from column 'created_at' that has DATETIME type
SELECT extract(year from created_at) as year,
       extract(month from created_at) as month, 
#Sum to add up all the sale_value. Round to make the data displayed in the table cleaner.
       sum(ROUND(sale_price,2)) as sale_value

FROM `bigquery-public-data.thelook_ecommerce.order_items`
#add parameter where status is complete and the dates are in Q1-Q3
WHERE status='Complete' and created_at BETWEEN '2022-01-01' AND '2022-10-01'
GROUP BY year,month
ORDER BY year,month ASC
'''
).to_dataframe()

#Display dataframe
orders_value

,year,month,sale_value
0,2022,1,73970.11
1,2022,2,67969.08
2,2022,3,81795.00
3,2022,4,82989.43
4,2022,5,89865.44
5,2022,6,90493.24
6,2022,7,109779.62
7,2022,8,118612.15
8,2022,9,127514.70


My Analysis: 



---

Same as the order count there has been an steady upward increase in sale_value over the years. This shows that the ecommerce store is growing and is in a healthy state. 

In the early month of 2022 the growth was slow but the rate of growth drastically increases as it enters month 5.

### 3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

In [12]:
customer_count = client.query(
'''
#Extract Year and month from column 'created_at' that has DATETIME type
SELECT extract(year from created_at) as year,
       extract(month from created_at) as month, 
#We use the Distinct method because the able 'Orders' has rows with the same user_id. Why? Users can make multiple orders.
       count(distinct(user_id)) as user_count

FROM `bigquery-public-data.thelook_ecommerce.orders`
#add parameter where status is complete and the dates are in Q1-Q3
WHERE status='Complete' and created_at BETWEEN '2022-01-01' AND '2022-10-01'
GROUP BY year,month
ORDER BY year,month ASC
'''
).to_dataframe()
#Display dataframe
customer_count


,year,month,user_count
0,2022,1,818
1,2022,2,800
2,2022,3,940
3,2022,4,919
4,2022,5,1016
5,2022,6,1077
6,2022,7,1297
7,2022,8,1308
8,2022,9,1448


My Analysis: 



---

The amount of users placing order fluctuates in the early month but in the late months the growth became steady with no decrease in user_count. This usually means that customers are satisfied with their order and are placing repeating orders in the following months.

### 4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

Read the commented code to better understand the meaning/reasoning behind each line of code. To make it easier to understand I divide the code into 2 section.

`Note:` Although we havent learned CTE I have tried several different ways to type the code but experienced issue (The code wont run, I can display the category count but cannot display the category name)

In [13]:
popular_category = client.query(
'''

##Section 1 
#Defines a Common Table Expression (CTE) called monthly_counts. 
WITH monthly_counts AS (
#Joins the order_items and inventory_items tables and selects the product_category, category_count and month (extracted from the created_at column) for all orders in q1-q3 2022 and Status = Complete.
  SELECT 
    inventory_items.product_category, 
    COUNT(*) AS category_count,
    extract(month from order_items.created_at) AS month
  FROM 
    bigquery-public-data.thelook_ecommerce.order_items AS order_items
    INNER JOIN bigquery-public-data.thelook_ecommerce.inventory_items AS inventory_items 
    ON order_items.product_id = inventory_items.product_id
  WHERE 
    order_items.status = 'Complete' 
    AND extract(month from order_items.created_at) BETWEEN 1 AND 9 
    AND extract(year from order_items.created_at) = 2022
  GROUP BY 
    inventory_items.product_category, 
    extract(month from order_items.created_at)
)




##Section 2
SELECT 
#Selects the month, product_category, and category_count columns from the monthly_counts CTE
  month, 
  product_category, 
  category_count
FROM (
  SELECT 
    month, 
    product_category, 
    category_count, 
#a ROW_NUMBER() function is used to assign a row number to each record within each month
#PARTITION BY, we can easily break down the results by a specific column (in this case, month), and perform operations on each group separately. 
#We use descending order, so that the top product category in each partition (i.e., the product category with the highest count) will be assigned a row number of 1.
    ROW_NUMBER() OVER (PARTITION BY month ORDER BY category_count DESC) AS row_num
  FROM 
    monthly_counts
    ORDER BY 
  month ASC
)
#WHERE clause to select only rows where row_num equals 1, we are filtering the results to show only the top product category for each month. 
WHERE 
  row_num = 1;


'''

).to_dataframe()
#Display dataframe
popular_category


,month,product_category,category_count
0,1,Jeans,1810
1,2,Tops & Tees,1779
2,3,Jeans,2313
3,4,Shorts,1992
4,5,Fashion Hoodies & Sweatshirts,2332
5,6,Tops & Tees,2305
6,7,Fashion Hoodies & Sweatshirts,2748
7,8,Jeans,2806
8,9,Jeans,2795


My Analysis: 



---

Overall, Jeans was the most popular product category in Q1-Q3 of 2022. This could mean that there is a increase in trend of jean products. Fashion Hoodies & Sweatshirts became the most popular product_category in months 5 & 7. This could mean that the category is seasonal and occurs in the summer months.

### 5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

Because I already know the best selling product category each month, to make the code simpler, I use the `Case` method. 

In [14]:
popular_category_value = client.query(
'''
SELECT
  extract(month from order_items.created_at) AS month,
  extract(year from order_items.created_at) AS year,
#Use a CASE statement to conditionally sum the sale_price column for specific months and product categories.
  SUM(CASE 
        WHEN extract(month from order_items.created_at) = 1 AND inventory_items.product_category = 'Jeans' THEN order_items.sale_price
        WHEN extract(month from order_items.created_at) = 2 AND inventory_items.product_category = 'Tops & Tees' THEN order_items.sale_price
        WHEN extract(month from order_items.created_at) = 3 AND inventory_items.product_category = 'Jeans' THEN order_items.sale_price
        WHEN extract(month from order_items.created_at) = 4 AND inventory_items.product_category = 'Shorts' THEN order_items.sale_price
        WHEN extract(month from order_items.created_at) = 5 AND inventory_items.product_category = 'Fashion Hoodies & Sweatshirts' THEN order_items.sale_price
        WHEN extract(month from order_items.created_at) = 6 AND inventory_items.product_category = 'Tops & Tees' THEN order_items.sale_price
        WHEN extract(month from order_items.created_at) = 7 AND inventory_items.product_category = 'Fashion Hoodies & Sweatshirts' THEN order_items.sale_price
        WHEN extract(month from order_items.created_at) = 8 AND inventory_items.product_category = 'Jeans' THEN order_items.sale_price
        WHEN extract(month from order_items.created_at) = 9 AND inventory_items.product_category = 'Jeans' THEN order_items.sale_price
        ELSE 0
      END) AS total_sum
#Joins the order_items and inventory_items tables using the product_id column.
FROM
  bigquery-public-data.thelook_ecommerce.order_items AS order_items
  INNER JOIN bigquery-public-data.thelook_ecommerce.inventory_items AS inventory_items 
    ON order_items.product_id = inventory_items.product_id

#Filters the data to include only rows where the created_at date is between January 1 and October 1 of 2022. 
WHERE 
  extract(month from order_items.created_at) BETWEEN 1 AND 9 
  AND extract(year from order_items.created_at) = 2022
GROUP BY 
  month,
  year
ORDER BY 
  month ASC;

'''
).to_dataframe()
#Display dataframe
popular_category_value


,month,year,total_sum
0,1,2022,7.552488e+05
1,2,2022,2.673854e+05
2,3,2022,8.575314e+05
3,4,2022,2.826745e+05
4,5,2022,4.127530e+05
5,6,2022,3.563164e+05
6,7,2022,5.377283e+05
7,8,2022,1.263155e+06
8,9,2022,1.123630e+06


My Analysis: 



---

We can see a steep drop in the sales of the most popular category in month 1-2. This is also seen in the total sale_value in question 2. The increase in month 1 can be caused by the new year holiday season. In month 2 when the holiday stops the sales decrease. The sales value in month 3-4 also sees a decrease in total sum , but the total sale_value didnt decrease. This indicates that the total_value is divide more equally towards the different categories.

###6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)

In [15]:
#Drop colums we wont be using
df1 = orders_value.drop(['year', 'month'], axis=1)
df2 = customer_count.drop(['year', 'month'], axis=1)

#Merge the 2 dataframes
combined_df = pd.merge(df1, df2, left_index=True, right_index=True)

#Print the correlation with the `.cor()' method
print(combined_df.corr())


            sale_value  user_count
sale_value    1.000000    0.990524
user_count    0.990524    1.000000


In [16]:
#Drop colums we wont be using
df3 = complete_orders.drop(['year', 'month'], axis=1)
df2 = customer_count.drop(['year', 'month'], axis=1)

#Merge the 2 dataframes
combined_df = pd.merge(df3, df2, left_index=True, right_index=True)

#Print the correlation with the `.cor()' method
print(combined_df.corr())

              order_amount  user_count
order_amount      1.000000    0.999955
user_count        0.999955    1.000000


My Analysis: 



---



A higher correlation between order_amount and user_count indicates a stronger relationship between the number of orders and the number of customers than the relationship between sales value and user_count. This may suggest that the customers of the e-commerce store may be placing more orders with smaller values rather than fewer orders with larger values.



Alternatively, it could also indicate that the store has a large customer base that is making many small purchases, which could be due to a variety of factors such as marketing campaigns, sales promotions, or the nature of the products being sold.

### IV.ANALISIS POINT / POIN ANALYSIS

###1. Berikan kesimpulan dari laporan/informasi yang dibuat berdasarkan problem statement/poin penjabaran menggunakan bahasa awam! (boleh dalam beberapa paragraf)

Before we look deeper into the transaction_value in the holiday period, here is a brief summary of the overall transaction,sale value and user count in our platform.

1.   Overall transaction value has increased over the last 3 quarters. This indicates that our company is `experiencing growth in terms of the number of transactions being made on the platform.`

2.   Total sales value has also increased. This suggests that our company is generating more revenue from these transactions, and that `customers are willing to spend more on the platform.`

3. The number of users on the platform has steadily increased. This indicates that our `company is attracting new customers and retaining existing ones`, which is an `important factor for long-term success`.


Now as you can see in the most popular product category table, we can see that the `Fashion Hoodies & Sweatshirts` became the most popular product category in month 5&7. This is around the summer holiday.

I would reccomend the marketing team to allocate their budget on the Fashion Hoodies & Sweatshirt category to promote during the summer holidays. This is also supported by the total sales value of `Fashion Hoodies & Sweatshirts` in month 5-7. We can see an increase in sales value for the most popular category compared to the month before the summer holiday.

This means that the ecommerce store makes most of its revenue from that category in the summer holiday months.

### V.QUESTION POINT / POIN PERTANYAAN

###1.Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

My Answer: 



---

Yes the problem statement can be measured because we have a time column. We can repeat the same data wrangling process in Q1-Q3 of 2023 and see if there are any difference. One Value we focus on to see if we hit the goal is the amount of average transaction value per customer. 

`Average Transaction Value per Transaction (AVT) = Sum of Sales Value / Transaction`


###2.Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?

In [22]:
# calculate percentage change in monthly average sale value
pct_change = orders_value['sale_value'].pct_change()

# drop the first row with null value
pct_change = pct_change.dropna()

pct_change

1   -0.081128
2    0.203415
3    0.014603
4    0.082854
5    0.006986
6    0.213125
7    0.080457
8    0.075056
Name: sale_value, dtype: float64

In [24]:
# calculate the mean of percentage change to get the average percentage increase month to month
avg_pct_change = pct_change.mean()

avg_pct_change

0.07442097876579766

My Answer: 



---
No, it is unlikely for the ecommerce store to hit` $250,000` in total sale value by the start of Q4. If we take the average increase per month of Q1-Q3 2022, an increase of `7.44%` from `$127,514` in end of Q3 is only `$137,001`.


###3.CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

In [25]:
# Divide the two columns and create a new dataframe with the result
df4 = pd.DataFrame({'ATV': orders_value['sale_value'] / complete_orders['order_amount']})

print(df4)

         ATV
0  90.207451
1   84.96135
2  86.464059
3  89.524736
4  87.930959
5  83.327109
6  83.929373
7  90.062377
8  86.981378


My Answer: 



---


Based on the data I have processed,In my opinion, company is eligible for new investors. Here are some of the reasons:


1.   Overall transaction value has increased over the last 3 quarters. This indicates that our company is `experiencing growth in terms of the number of transactions being made on the platform.`

2.   Total sales value has also increased. This suggests that our company is generating more revenue from these transactions, and that `customers are willing to spend more on the platform.`

3. Average transaction value (ATV) has remained stable at around $85 per transaction. This means that the growth in totale sales value is not only due to price increases or higher-value transactions, but rather represents `organic growth.`

4. The number of users on the platform has steadily increased. This indicates that our `company is attracting new customers and retaining existing ones`, which is an `important factor for long-term success`.

5. Finally, the average increase in transaction value per month is 7.44%. This is a `healthy growth rate` that suggests our company is well-positioned to continue its growth.
